In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [14]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy

def load_kernel_matrix_from_splits(base_path, dataset_size, size_splits, number_splits, file_name_start="NTK_"):

  l = lambda x : x*size_splits 
  h = lambda x : (1+x)*size_splits if (not x == number_splits) else dataset_size-1

  kernel_matrix = np.empty((dataset_size, dataset_size))

  for i in range(number_splits+1):
    for j in range(number_splits+1):    
      split_k = np.load(kernel_path + f"/NTK_{h(i)}_{h(j)}.npy")
      kernel_matrix[l(i):h(i),l(j):h(j)] = split_k
    
  return kernel_matrix


def kernel_SVM(kernel_matrix, ys):
  C_list = np.logspace(-2, 4, 120)
  svc = SVC(kernel='precomputed', max_iter=5000)
  clf = GridSearchCV(svc, {'C' : C_list}, verbose=0, return_train_score=True)
  clf.fit(kernel_matrix, ys)

  return clf, pd.DataFrame(clf.cv_results_)

In [3]:
# GCNN data
data_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG"
kernel_path = f"/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/GCNN"

kernel_matrix = load_kernel_matrix_from_splits(kernel_path, 188, 50, 3)
ys = np.load(data_path + f"/ys.npy")

kernel_matrix = kernel_matrix[:-1,:-1]
ys = ys[:-1]
ys = np.squeeze(ys, 1)

In [12]:
kernel_matrix[-5:,-5:]

array([[  73008.,   72072.,  393120.,  393120.,   88920.],
       [  72072.,   71148.,  388080.,  388080.,   87780.],
       [ 393120.,  388080., 2116800., 2116800.,  478800.],
       [ 393120.,  388080., 2116800., 2116800.,  478800.],
       [  88920.,   87780.,  478800.,  478800.,  108300.]])

In [15]:
# estimate kernel svm
svc, result = kernel_SVM(kernel_matrix, ys)
result[result["mean_test_score"] == result["mean_test_score"].max()]

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
39,0.00502,0.000183,0.001222,0.000044,0.925522,{'C': 0.9255215909791353},0.921053,0.921053,0.837838,0.864865,...,0.876529,0.037669,1,0.845638,0.818792,0.866667,0.86,0.866667,0.851553,0.018091


In [21]:
from sklearn.kernel_ridge import KernelRidge
import numpy as np

krr = KernelRidge(alpha=1.0, kernel="precomputed")
krr.fit(kernel_matrix, ys)

np.sum(np.array(krr.predict(kernel_matrix) < 0.5, dtype="int") == ys) / 188

array([1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

In [ ]:
svc = SVC(kernel='precomputed', max_iter=5e5, C=6285.203269)
svc.fit(kernel_matrix, ys)
svc.predict(kernel_matrix)

array([1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1], dtype=int32)

In [16]:
# Two_wl Kernel
data_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG"
kernel_path = f"/content/drive/MyDrive/MasterarbeitData/Kernels/MUTAG/TWO_WL"

kernel_matrix = load_kernel_matrix_from_splits(kernel_path, 188, 10, 18)
ys = np.load(data_path + f"/ys.npy")

kernel_matrix = kernel_matrix[:-1,:-1]
ys = ys[:-1]
ys = np.squeeze(ys, 1)

# estimate kernel svm
svc, result = kernel_SVM(kernel_matrix, ys)
result[result["mean_test_score"] == result["mean_test_score"].max()]

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
5,0.001715,0.000306,0.000671,0.000015,0.017869,{'C': 0.017869024337358284},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.838926,0.838926,0.846667,0.846667,0.846667,0.843570,0.003792
6,0.001662,0.000167,0.000674,0.000015,0.020069,{'C': 0.020068781676649675},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.838926,0.838926,0.840000,0.846667,0.840000,0.840904,0.002921
7,0.001532,0.000026,0.000662,0.000018,0.022539,{'C': 0.022539339047347912},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.838926,0.838926,0.840000,0.840000,0.840000,0.839570,0.000526
8,0.001690,0.000113,0.000703,0.000029,0.025314,{'C': 0.02531403315241567},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.838926,0.838926,0.840000,0.840000,0.840000,0.839570,0.000526
9,0.001681,0.000184,0.000777,0.000167,0.02843,{'C': 0.028430304593026635},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.838926,0.838926,0.840000,0.840000,0.840000,0.839570,0.000526
10,0.001560,0.000011,0.000667,0.000016,0.03193,{'C': 0.031930203076910274},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.838926,0.838926,0.840000,0.840000,0.840000,0.839570,0.000526
11,0.001552,0.000044,0.000663,0.000012,0.035861,{'C': 0.03586095482011831},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.838926,0.825503,0.840000,0.840000,0.840000,0.836886,0.005706
12,0.001528,0.000024,0.000657,0.000007,0.040276,{'C': 0.040275599798502974},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.832215,0.838926,0.840000,0.833333,0.840000,0.836895,0.003406
13,0.001510,0.000029,0.000652,0.000006,0.045234,{'C': 0.04523370744772101},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.832215,0.832215,0.833333,0.833333,0.840000,0.834219,0.002933
14,0.001577,0.000084,0.000676,0.000012,0.050802,{'C': 0.05080218046913023},0.868421,0.815789,0.837838,0.864865,...,0.84495,0.019488,1,0.832215,0.818792,0.833333,0.840000,0.840000,0.832868,0.007753
